<a href="https://colab.research.google.com/github/caramelpopcornKHU/caramelpopcornKHU/blob/main/network%EC%A4%91%EC%8B%AC%EC%84%B1%EA%B3%84%EC%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import networkx as nx
import osmnx as ox

In [ ]:
# 설정
ox.settings.use_cache = True

In [ ]:
# 도로망 그래프 다운로드
#G = ox.graph_from_point((37.79, -122.41), dist=750, network_type="drive")
G = ox.graph.graph_from_place('서울특별시', network_type = 'drive')

In [ ]:
# 속도 및 소요 시간 계산
korea_speed_defaults = {
    "motorway": 100,       # 고속도로
    "trunk": 80,           # 자동차 전용도로, 국도
    "primary": 60,         # 주요 간선도로
    "secondary": 50,       # 보조 간선도로
    "tertiary": 40,        # 연결도로
    "residential": 30,     # 주택가 도로
    "service": 20          # 골목길, 주차장 진입로
}


G = ox.add_edge_speeds(G , hwy_speeds = korea_speed_defaults, fallback=30)  # 한국 시도 조례 기준 속도 가정, 정보가 없는 도로는 30키로
G = ox.add_edge_travel_times(G)

In [ ]:
# GeoDataFrame으로 변환 및 다시 그래프로 재구성
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
G = ox.graph_from_gdfs(gdf_nodes, gdf_edges, graph_attrs=G.graph)

In [ ]:
D = nx.DiGraph()
for u, v, data in G.edges(data=True):
    existing_data = D.get_edge_data(u, v)
    if existing_data is None:
        D.add_edge(u, v, **data)
    else:
        if data.get("travel_time", float("inf")) < existing_data.get("travel_time", float("inf")):
            # 먼저 edge를 삭제하고 다시 추가
            D.remove_edge(u, v)
            D.add_edge(u, v, **data)


In [ ]:
# 중심성 계산 (travel_time 가중치 사용)
bc = nx.betweenness_centrality(D, weight="travel_time", normalized=True)
nx.set_node_attributes(G, bc, "bc")


In [ ]:
# 시각화
nc = ox.plot.get_node_colors_by_attr(G, "bc", cmap="plasma")
fig, ax = ox.plot_graph(
    G,
    bgcolor="k",
    node_color=nc,
    node_size=50,
    edge_linewidth=2,
    edge_color="#333333"
)